<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/Strato75/Covid-19_ItalyStats/blob/master/notebooks/per_region_stats.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" />Run in Google Colab</a>
  </td>
</table>

In [ ]:
%pylab inline
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates 

In [ ]:
## Globals
x_tick_fs = 6

## Functions
def plot_field(groups, sel_regions, field, ms = 4):
    ax0 = plt.gca()
    for g in groups.groups:

        #x_data = ['-'.join(i.split('T')[0].split('-')[1:]) for i in np.datetime_as_string(groups.get_group(g)['data'].values)]  
        x_data = groups.get_group(g)['data'].values
        data = groups.get_group(g)[field]

        m = max(data)
        if g in sel_regions:
            plt.plot(x_data, data, 'o-', ms=ms, label='%s, max:%d, curr:%d' %(g, m, data.iloc[-1]))

    plt.title(field)
    plt.grid()
    plt.xlabel('Data')
    plt.ylabel('# casi')
    #_ = plt.xticks(rotation=90, fontsize=x_tick_fs)
    _ = ax0.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d')) 
    _ = ax0.xaxis.set_major_locator(plt.MaxNLocator(40))
    _ = plt.setp(ax0.get_xticklabels(), rotation = 90, fontsize=x_tick_fs)
    plt.legend(loc='upper left')


def get_df_from_json_url(url):
    response = requests.get(url)
    #print(response.text)
    df = pd.read_json(response.text, orient='records')
    return df

In [ ]:
raw_it_region_df = get_df_from_json_url('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-json/dpc-covid19-ita-regioni.json') 
raw_it_region_df['data'] = pd.to_datetime(raw_it_region_df['data'], format='%Y-%m-%dT%H:%M:%S')

### Assign the _minval_ variable to set the number of cases a region must have to plot its statistics

In [ ]:
minval = 5000 

In [ ]:
groups_regions = raw_it_region_df.groupby('denominazione_regione')

cols = raw_it_region_df.columns
fields = sorted(['ricoverati_con_sintomi', 'terapia_intensiva',
       'totale_ospedalizzati', 'isolamento_domiciliare', 'totale_positivi',
       'variazione_totale_positivi', 'nuovi_positivi', 'dimessi_guariti',
       'deceduti', 'totale_casi', 'tamponi'])

In [ ]:
sel_regions = list(raw_it_region_df.loc[raw_it_region_df['totale_casi'] > minval, 'denominazione_regione'].unique())

f = plt.figure(figsize=(22,20))

for index, field in enumerate(fields):
    plt.subplot(4,3,index+1)
    plot_field(groups_regions, sel_regions, field)

plt.tight_layout(rect=[0, 0.01, 1, 0.95])
plt.suptitle("Statistiche per regione (regioni con più di %d casi totali)" % minval, fontsize=18)

f.savefig('figures/per_region_stats.png', dpi=300, transparency=False, bbox_inches='tight')